In [0]:
%run "/Shared/CP_Data/Config/config"

In [0]:
%run "/Shared/CP_Data/Shared_script/main"

In [0]:
from pyspark.sql.functions import max

In [0]:
try:
    dbutils.widgets.text('source_name', '', 'Source');
    dbutils.widgets.text('source_type', '', 'Source Type');
    dbutils.widgets.text('dataset', '', 'Dataset');
    dbutils.widgets.text('land_file_path', '', 'Land File Path');
    dbutils.widgets.text('land_file_name', '', 'Land File Name');
    dbutils.widgets.text('inc_column_name', '', 'Inc Column Name');
    dbutils.widgets.text('HWM_Query', '', 'HWM Query');
    dbutils.widgets.text('last_load_value','','Last Load Value')
except Exception as e:
  print(str(e))
  raise Exception(e)

In [0]:
try:
    dataset = dbutils.widgets.get('dataset');
    vLandFilePath = dbutils.widgets.get('land_file_path');
    vLandFileName = dbutils.widgets.get('land_file_name');
    delta_key = dbutils.widgets.get('inc_column_name');
    HWMQuery = dbutils.widgets.get('HWM_Query');
    LastLoadValue = dbutils.widgets.get('last_load_value');
except Exception as e:
  print(str(e))
  raise Exception(e)

In [0]:
vADLSMountBase = '/mnt/ADLS/'
vLandFilePath = vADLSMountBase + vLandFilePath + '/' + vLandFileName
dfLand = spark.read.format("avro").load(vLandFilePath)

In [0]:
try:
  last_load_value=None
  # Check if delta_key is null...
  if len(delta_key) == 0:
    print("This is a Full load dataset...")
    if dfLand.count() > 0:
      print("Land File is not Empty")    
    elif dfLand.count() == 0:
      print("Land File is empty")
  
  elif len(delta_key) > 0:
    print("This is an SCD_2 type dataset....")
    if dfLand.count() > 0:
      print("Land file is not Empty")
      dfLand.createOrReplaceTempView(dataset)
      # Get the max(delta_key) as store it as HWM column value...
      incLoadQuery = HWMQuery
      last_load_value = spark.sql(incLoadQuery).first()[0]   # Get the HWM Value from Land File....
    
    elif dfLand.count() == 0 and LastLoadValue is not None:
      print("Land File is Empty for SCD_2 ingestion")
      last_load_value = LastLoadValue

except Exception as ex:
  print(str(ex))
  raise Exception(ex)

In [0]:
# dfLand.createOrReplaceTempView(dataset)

In [0]:
# last_load_value = spark.sql(HWMQuery).first()[0]
# print(last_load_value)

In [0]:
dbutils.notebook.exit(json.dumps({'status': 'Succeeded', 'CurrentHWM': last_load_value, 'Message': successMessage}))